## Exploring housing markers in Canada and British Columbia


Average value of new mortgage loan 

Canada, provinces and CMAs 2012 Q3 to 2020 Q3

In [1]:
%run ./scr/main.py
import plotly.express as px

In [2]:
data = "./data/"

In [ ]:
# a function for data cleaning
def data_prep(data_file):
    temp = pd.read_excel(data_file)
    
    # Resetting column
    new_header = temp.iloc[3]
    temp = temp[4:]
    temp.columns = new_header



#### Data contains quite a bit of additional information

In [4]:
average_new_mortgage = pd.read_excel(data + "average-value-new-mortgage-loans-ca-prov-cmas-2012-q3-2020-q3-en.xlsx")
# Resetting column
new_header = average_new_mortgage.iloc[3]
average_new_mortgage = average_new_mortgage[4:]
average_new_mortgage.columns = new_header
#Removing empty rows
average_new_mortgage.dropna(thresh=2, inplace=True)
#Ensuring the indices go from 0 without skipping any number
average_new_mortgage.reset_index(inplace=True)
average_new_mortgage.drop(['index'],inplace=True,axis=1)
#Selecting all rows until and including the row for "Victoria"
row_select = average_new_mortgage[average_new_mortgage['Geography']=='Victoria'].index
average_new_mortgage = average_new_mortgage.iloc[:row_select[0]+1,1:]

Cleanup of rows

Data needs a fair amount of reformatting.

In [6]:
#Categorize Geography as Country, Province, and City
average_new_mortgage['Region/City'] = ""
average_new_mortgage['Region/City'][average_new_mortgage.index ==0] = 'Country'
average_new_mortgage['Region/City'][(average_new_mortgage.index < 11) & (average_new_mortgage.index >0)] = 'Province'
average_new_mortgage['Region/City'][average_new_mortgage.index >= 11] = 'City'

Rearrange columns

In [8]:
#Rearrange columns -- bring column 'Region/City' to the 1st column position
# Access columns
cols = average_new_mortgage.columns.tolist()
# Bring last col to front
cols = cols[-1:] + cols[:-1]
# Set new column order
average_new_mortgage = average_new_mortgage[cols] 

Let's begin data exploration.

In [ ]:
average_new_mortgage.info()

Data types are incorrect. Let's convert to the right format.

In [ ]:
#Isolate yearly and quarterly columns and convert to numeric.
int_cols = average_new_mortgage.columns.drop(['Region/City', 'Geography'])
average_new_mortgage[int_cols] = average_new_mortgage[int_cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
average_new_mortgage.info()

Let's do summary statistics.

In [ ]:
average_new_mortgage.describe()

In [ ]:
df_province = average_new_mortgage[average_new_mortgage['Region/City']=="Province"]

In [ ]:
df_province.iloc[:,2:].plot(kind='bar')

In [ ]:
def graph_region(df, level: str, graph_type: str):
    """
    df: data frame object with mortage data
    level: "Province" or "City"
    graph_type: "box", "violin"
    """
    
    plot_dict = {'box': px.box,'violin': px.violin}
    
    def slice_data(df, level):
        """
        Extract a subset of df based on level
        Return a dataframe
        """
        temp = df[df['Region/City']==level]
        temp = pd.melt(temp, id_vars='Geography', value_vars=temp.columns[2:])
        temp.rename(columns = {3:'Time'}, inplace = True)
        return temp
    
    try:
        region_df = slice_data(df, level)
        fig = plot_dict[graph_type](region_df, x="Geography", y="value", color = "Geography", points='all')
        fig.update_xaxes(tickangle=-45)
        fig.show()
    
    except KeyError: print("Key not found. Make sure that 'level' is in ['Province','City']", 
                           "and 'graph_type' is in ['box','violin']")


In [ ]:
graph_region(average_new_mortgage, level='Province', graph_type='box')

In [ ]:
#line plot
fig = px.line(temp, x='Time', y='value', color = 'Geography')
fig.update_xaxes(tickangle=-45)
fig.show()

## Exercise

1. Identify different ways of visualizing data. 
2. What insights do you gain for the study period? 
3. How are these insights different for each province/city?